In [49]:
import glob
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import csv

In [12]:
def clean_html(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [25]:
with open("spam-mail.tr.label", "r") as file:
    lines = file.readlines()
    
lines = [l.strip() for l in lines[1:]]
labels = {}
for line in lines:
    x = int(line.split(',')[0])
    y = int(line.split(',')[1])
    labels[x] = y

In [39]:
email_folder = "tr"
emails = glob.glob(email_folder + "/*")

train_x = []
train_y = []
test_x = []
test_y = []

for email in emails[:int(0.8* len(emails))]:
    with open(email, "r") as handle:
        email_html = handle.read()
        email_clean = clean_html(email_html)
    
    train_x.append(email_html)
    id = int(email.split('.')[0].split('_')[1])
    train_y.append(labels[id])

for email in emails[int(0.8* len(emails)):]:
    with open(email, "r") as handle:
        email_html = handle.read()
        email_clean = clean_html(email_html)
    
    test_x.append(email_html)
    id = int(email.split('.')[0].split('_')[1])
    test_y.append(labels[id])

In [40]:
count_vect = CountVectorizer()
train_x_count = count_vect.fit_transform(train_x)

In [41]:
clf = MultinomialNB().fit(train_x_count, train_y)

In [45]:
test_x_count = count_vect.transform(test_x)
predicted = clf.predict(test_x_count)

count_true = 0
count_false = 0
for pred, label in zip(predicted, test_y):
    if pred == label:
        count_true += 1
    else:
        count_false += 1

In [51]:
print(classification_report(y_true= test_y, y_pred= predicted))

             precision    recall  f1-score   support

          0       0.99      0.79      0.88       160
          1       0.91      1.00      0.95       339

avg / total       0.94      0.93      0.93       499

